## Fine-Tuning Language Model 💻

In [10]:
# Setup Labels for Download XLM Roberta Base
labels_to_id = {
    "O": 0,
    "B-DRUG_NAME": 1, "I-DRUG_NAME": 2,
    "B-DOSAGE": 3, "I-DOSAGE": 4,
    "B-FORM": 5, "I-FORM": 6,
    "B-WARNINGS": 7, "I-WARNINGS": 8,
    "B-INDICATIONS": 9, "I-INDICATIONS": 10,
    "B-USAGE_INSTRUCTIONS": 11, "I-USAGE_INSTRUCTIONS": 12
}
id_to_labels = {v: k for k, v in labels_to_id.items()}

In [11]:
# XLM Roberta Base - Pretrained Model Download From HuggingFace
from transformers import XLMRobertaForTokenClassification
xlm_roberta_model = XLMRobertaForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=len(labels_to_id), id2label=id_to_labels, label2id=labels_to_id)

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Custom Tokenizer - Tokenizer Download From HuggingFace
from transformers import XLMRobertaTokenizer
xlm_roberta_tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

In [13]:
# Resize Embeddings of Model
xlm_roberta_model.resize_token_embeddings(len(xlm_roberta_tokenizer))

Embedding(250002, 768, padding_idx=1)

In [14]:
# Import Custom Tokenizer Dataset to fine-tuning
import json
with open('../data/xlm_roberta_tokenizer_format.json', 'r', encoding='utf-8') as f:
    xlm_roberta_tokenizer_dataset = json.load(f)

In [15]:
# Mapping dataset
from datasets import Dataset
xlm_roberta_tokenizer_dataset = Dataset.from_dict({
    "tokens": [item["tokens"] for item in xlm_roberta_tokenizer_dataset],
    "labels": [item["ner_tags"] for item in xlm_roberta_tokenizer_dataset]
})
print(xlm_roberta_tokenizer_dataset)

Dataset({
    features: ['tokens', 'labels'],
    num_rows: 2000
})


In [16]:
# Function to align dataset
def tokenize_and_align_labels(examples):
    tokenized_inputs = xlm_roberta_tokenizer(examples['tokens'], padding="longest", truncation=True, return_tensors="pt", is_split_into_words=True)
    aligned_labels = []
    for i, labels in enumerate(examples['labels']):
        label_ids = [labels_to_id[label] for label in labels]
        padding_length = len(tokenized_inputs['input_ids'][i]) - len(label_ids)
        label_ids += [-100] * padding_length
        aligned_labels.append(label_ids)
        
    tokenized_inputs['labels'] = aligned_labels
    return tokenized_inputs

In [17]:
# Align labels of Custom Tokenizer Dataset
xlm_roberta_tokenizer_dataset = xlm_roberta_tokenizer_dataset.map(tokenize_and_align_labels, batched=True)

Map: 100%|██████████| 2000/2000 [00:01<00:00, 1966.59 examples/s]


In [18]:
# Display number of XLM Roberta Base - Tokenizer Dataset
print(xlm_roberta_tokenizer_dataset)

Dataset({
    features: ['tokens', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 2000
})


In [19]:
# Setup Traning Arguments
train_dataset = xlm_roberta_tokenizer_dataset.select(range(0, 800))
eval_dataset = xlm_roberta_tokenizer_dataset.select(range(800, 1000))

from transformers import Trainer, TrainingArguments
training_arguments = TrainingArguments(
    output_dir='../output/model/original_tokenizer',         
    num_train_epochs=10,            
    per_device_train_batch_size=4,   
    per_device_eval_batch_size=4, 
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)
trainer = Trainer(
    model=xlm_roberta_model, 
    args=training_arguments, 
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=xlm_roberta_tokenizer
)
trainer.train()

/Users/waterondaway/Laboratory/medicine-scan-lm/.venv/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/xc/h44mkk7s0hx2k11xb082y6l00000gn/T/ipykernel_1462/1727958734.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.586000,0.519944
2,0.452800,0.385887
3,0.390500,0.366439
4,0.274400,0.290832
5,0.215500,0.239205
6,0.178200,0.248815
7,0.129600,0.168967
8,0.102600,0.163410
9,0.072900,0.196725
10,0.054600,0.176007


TrainOutput(global_step=2000, training_loss=0.2952613136768341, metrics={'train_runtime': 8364.8925, 'train_samples_per_second': 0.956, 'train_steps_per_second': 0.239, 'total_flos': 428732624880000.0, 'train_loss': 0.2952613136768341, 'epoch': 10.0})

In [12]:
# ใช้โมเดลและ tokenizer จาก trainer โดยตรง
model = trainer.model
tokenizer = trainer.tokenizer

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


In [33]:
print(model.config.id2label)

{0: 'O', 1: 'B-DRUG_NAME', 2: 'I-DRUG_NAME', 3: 'B-DOSAGE', 4: 'I-DOSAGE', 5: 'B-FORM', 6: 'I-FORM', 7: 'B-WARNINGS', 8: 'I-WARNINGS', 9: 'B-INDICATIONS', 10: 'I-INDICATIONS', 11: 'B-USAGE_INSTRUCTIONS', 12: 'I-USAGE_INSTRUCTIONS'}


In [ ]:
# CUSTOM TOKENIZER | 10 EPORCH | 1000 DATASET
from transformers import pipeline

# สร้าง NER pipeline
ner_pipeline = pipeline("ner", model='../output/model/custom_tokenizer/v2', tokenizer='../output/model/custom_tokenizer/v2', aggregation_strategy="simple")

# ทดสอบทำนาย
text = """รักษาผลกระทบจากการผ่าตัด รับประทานเมื่อมีอาการปวด"""
ner_results = ner_pipeline(text)
print(ner_results)

for entity in ner_results:
    print(f"{entity['word']} ({entity['entity_group']}): {entity['score']:.3f}")


Device set to use mps:0


[{'entity_group': 'USAGE_INSTRUCTIONS', 'score': 0.98609334, 'word': 'รักษาผลกระทบจากการผ่าตัด ', 'start': 0, 'end': 25}, {'entity_group': 'INDICATIONS', 'score': 0.680932, 'word': 'รับประทานเมื่อมีอาการปวด', 'start': 25, 'end': 49}]
รักษาผลกระทบจากการผ่าตัด  (USAGE_INSTRUCTIONS): 0.986
รับประทานเมื่อมีอาการปวด (INDICATIONS): 0.681


In [ ]:
# ORIGINAL TOKENIZER | 10 EPORCH | 1000 DATASET
from transformers import pipeline

# สร้าง NER pipeline
ner_pipeline = pipeline("ner", model='../output/model/original_tokenizer/checkpoint-2000', tokenizer='../output/model/original_tokenizer/checkpoint-2000', aggregation_strategy="simple")

# ทดสอบทำนาย
text = """รักษาผลกระทบจากการผ่าตัด รับประทานเมื่อมีอาการปวด ไม่เกิน 3 วัน"""
ner_results = ner_pipeline(text)
print(ner_results)

for entity in ner_results:
    print(f"{entity['word']} ({entity['entity_group']}): {entity['score']:.3f}")


Device set to use mps:0


[{'entity_group': 'USAGE_INSTRUCTIONS', 'score': 0.9908006, 'word': 'รักษาผลกระทบจากการผ่าตัด รับประทานเมื่อมีอาการปวด ไม่เกิน 3 วัน', 'start': 0, 'end': 63}]
รักษาผลกระทบจากการผ่าตัด รับประทานเมื่อมีอาการปวด ไม่เกิน 3 วัน (USAGE_INSTRUCTIONS): 0.991


In [73]:
# CUSTOM TOKENIZER | 10 EPORCH | 1000 DATASET
from transformers import pipeline

# สร้าง NER pipeline
ner_pipeline = pipeline("ner", model='../output/model/custom_tokenizer/v2', tokenizer='../output/model/custom_tokenizer/v2', aggregation_strategy="simple")

# ทดสอบทำนาย
text = """รับประทาน 10 เม็ดก่อนนอน"""
ner_results = ner_pipeline(text)
print(ner_results)

for entity in ner_results:
    print(f"{entity['word']} ({entity['entity_group']}): {entity['score']:.3f}")


Device set to use mps:0


[{'entity_group': 'INDICATIONS', 'score': 0.67953825, 'word': 'ก่อนนอน', 'start': 17, 'end': 24}]
ก่อนนอน (INDICATIONS): 0.680


In [78]:
# ORIGINAL TOKENIZER | 10 EPORCH | 1000 DATASET
from transformers import pipeline

# สร้าง NER pipeline
ner_pipeline = pipeline("ner", model='../output/model/original_tokenizer/checkpoint-2000', tokenizer='../output/model/original_tokenizer/checkpoint-2000', aggregation_strategy="simple")

# ทดสอบทำนาย
text = """รับประทาน 10 เม็ดก่อนนอน"""
ner_results = ner_pipeline(text)
print(ner_results)

for entity in ner_results:
    print(f"{entity['word']} ({entity['entity_group']}): {entity['score']:.3f}")


Device set to use mps:0


[{'entity_group': 'USAGE_INSTRUCTIONS', 'score': 0.94189125, 'word': 'รับประทาน 10 เม็ดก่อนนอน', 'start': 0, 'end': 24}]
รับประทาน 10 เม็ดก่อนนอน (USAGE_INSTRUCTIONS): 0.942


In [64]:
# CUSTOM TOKENIZER | 10 EPORCH | 1000 DATASET
from transformers import pipeline

# สร้าง NER pipeline
ner_pipeline = pipeline("ner", model='../output/model/custom_tokenizer/v2', tokenizer='../output/model/custom_tokenizer/v2', aggregation_strategy="simple")

# ทดสอบทำนาย
text = """ช่วยฟื้นฟูหลังการออกกำลังกาย รับประทาน 2 เม็ดก่อนนอน"""
ner_results = ner_pipeline(text)
print(ner_results)

for entity in ner_results:
    print(f"{entity['word']} ({entity['entity_group']}): {entity['score']:.3f}")

Device set to use mps:0


[{'entity_group': 'USAGE_INSTRUCTIONS', 'score': 0.9944506, 'word': 'ช่วยฟื้นฟูหลังการออกกําลังกาย ', 'start': 0, 'end': 29}, {'entity_group': 'INDICATIONS', 'score': 0.7897528, 'word': 'รับประทาน 2 เม็ดก่อนนอน', 'start': 29, 'end': 52}]
ช่วยฟื้นฟูหลังการออกกําลังกาย  (USAGE_INSTRUCTIONS): 0.994
รับประทาน 2 เม็ดก่อนนอน (INDICATIONS): 0.790


In [ ]:
# ORIGINAL TOKENIZER | 10 EPORCH | 1000 DATASET
from transformers import pipeline

# สร้าง NER pipeline
ner_pipeline = pipeline("ner", model='../output/model/original_tokenizer/checkpoint-1600', tokenizer='../output/model/original_tokenizer/checkpoint-1600', aggregation_strategy="simple")

# ทดสอบทำนาย
text = """ช่วยฟื้นฟูหลังการออกกำลังกาย รับประทาน 2 เม็ดก่อนนอน"""
ner_results = ner_pipeline(text)
print(ner_results)

for entity in ner_results:
    print(f"{entity['word']} ({entity['entity_group']}): {entity['score']:.3f}")


Device set to use mps:0


[{'entity_group': 'USAGE_INSTRUCTIONS', 'score': 0.95436054, 'word': 'ช่วยฟื้นฟูหลังการออกกําลังกาย รับประทาน 2 เม็ดก่อนนอน', 'start': 0, 'end': 52}]
ช่วยฟื้นฟูหลังการออกกําลังกาย รับประทาน 2 เม็ดก่อนนอน (USAGE_INSTRUCTIONS): 0.954


In [ ]:
# ORIGINAL TOKENIZER | 10 EPORCH | 1000 DATASET
from transformers import pipeline

# สร้าง NER pipeline
ner_pipeline = pipeline("ner", model='../output/model/original_tokenizer/checkpoint-2000', tokenizer='../output/model/original_tokenizer/checkpoint-2000', aggregation_strategy="simple")

# ทดสอบทำนาย
text = """
รับประทานก่อนอาหาร 30 นาที
ช่วยรักษาผื่นภูมิแพ้
หากมีอาการบวมที่ใบหน้าหรือริมฝีปาก ควรหยุดยา
"""
ner_results = ner_pipeline(text)
print(ner_results)

for entity in ner_results:
    print(f"{entity['word']} ({entity['entity_group']}): {entity['score']:.3f}")


Device set to use mps:0


[{'entity_group': 'USAGE_INSTRUCTIONS', 'score': 0.9677924, 'word': 'เม็ด รับประทานก่อนอาหาร 30 นาที ช่วยรักษาผื่นภูมิแพ้', 'start': 15, 'end': 67}, {'entity_group': 'INDICATIONS', 'score': 0.33644873, 'word': 'หาก', 'start': 67, 'end': 71}, {'entity_group': 'USAGE_INSTRUCTIONS', 'score': 0.46490714, 'word': 'มีอาการ', 'start': 71, 'end': 78}, {'entity_group': 'FORM', 'score': 0.6127798, 'word': 'บวม', 'start': 78, 'end': 81}]
เม็ด รับประทานก่อนอาหาร 30 นาที ช่วยรักษาผื่นภูมิแพ้ (USAGE_INSTRUCTIONS): 0.968
หาก (INDICATIONS): 0.336
มีอาการ (USAGE_INSTRUCTIONS): 0.465
บวม (FORM): 0.613


In [79]:
# CUSTOM TOKENIZER | 10 EPORCH | 1000 DATASET
from transformers import pipeline

# สร้าง NER pipeline
ner_pipeline = pipeline("ner", model='../output/model/custom_tokenizer/v2', tokenizer='../output/model/custom_tokenizer/v2', aggregation_strategy="simple")

# ทดสอบทำนาย
text = """
รับประทานก่อนอาหาร 30 นาที
ช่วยรักษาผื่นภูมิแพ้
หากมีอาการบวมที่ใบหน้าหรือริมฝีปาก ควรหยุดยา
"""
ner_results = ner_pipeline(text)
print(ner_results)

for entity in ner_results:
    print(f"{entity['word']} ({entity['entity_group']}): {entity['score']:.3f}")


Device set to use mps:0


[{'entity_group': 'INDICATIONS', 'score': 0.6580069, 'word': 'รับประทาน', 'start': 0, 'end': 10}, {'entity_group': 'USAGE_INSTRUCTIONS', 'score': 0.4204036, 'word': 'ก่อน', 'start': 10, 'end': 14}, {'entity_group': 'USAGE_INSTRUCTIONS', 'score': 0.86365354, 'word': 'อาหาร 30 นาที ช่วยรักษาผื่น', 'start': 14, 'end': 41}, {'entity_group': 'FORM', 'score': 0.63934684, 'word': 'ภูมิแพ้', 'start': 41, 'end': 48}, {'entity_group': 'USAGE_INSTRUCTIONS', 'score': 0.5501631, 'word': 'หากมีอาการบวมที่ใบหน้า', 'start': 48, 'end': 71}, {'entity_group': 'USAGE_INSTRUCTIONS', 'score': 0.41016936, 'word': 'ปาก ควร', 'start': 80, 'end': 87}]
รับประทาน (INDICATIONS): 0.658
ก่อน (USAGE_INSTRUCTIONS): 0.420
อาหาร 30 นาที ช่วยรักษาผื่น (USAGE_INSTRUCTIONS): 0.864
ภูมิแพ้ (FORM): 0.639
หากมีอาการบวมที่ใบหน้า (USAGE_INSTRUCTIONS): 0.550
ปาก ควร (USAGE_INSTRUCTIONS): 0.410
